# Homework 3

# Problem Description

In this exercise we implement and test the Gaussian discriminant analysis, as
described in Sec 9.2 in Murphy's book. The project can be broken into three tasks
(1) Generate the data points for all classes $c\in\{0,1,2,..,C\}$, and compute
the approximations $\mu_c$ and $\sigma_c$ that are based on the data points. Also,
generate a second set of points, with known tlabels, hat will be used to test
the classification function.
(2) Evaluate the multivariate Gaussian distribution on the test points of all
classes to obtain the probabilities p(X = tjY = c), where t runs through
all test points and c runs through all classes.
(3) Compute the probabilities p(Y = cjX = t) on the test points using Bayes'
formula and for every test point find the value of c with the maximal prob-
ability.
In your implementation, you should write a separate subroutine for each task. These
routines should be written such that they can be used for any dimension D and
any number of classes C.
You should first experiment with artificially generated data. To that end, use the
numpy routine numpy.random.multivariate_normal that generates points ran-
domly by the multivariate Gauss distribution. In that case, the subroutine for task
1 has an interface as follows
#
- Parameters
- mean center of the Gaussian
- cov covariance matrix
- nx number data points
- nt number test points
- Returns
- x set of random vectors of length nx (data points)
- t set of random vectors of length nt (test points)
- mu mean based on x
- Sgm covariance matrix based on x


#
def getArtificialData(mean, cov, nx, nt):

Note that this routine calls the random.multivariate_normal twice, i.e., for the
data and test set. The vector mean and the matrix cov must be in $R^D$ and $R^{D\times D}$.
The returned values mu and Sgm are computed by (9.19) and (9.20) in Murphy's
book. They should be similar to mean and cov, but are different, in general. This
routine must be called for each class c. To get the test points of each class into one
long vector use the numpy.concatenate function.

\\

The interface for the second task should look like this
#
- Parameters
- t points: t[n,d] is the d-th component of the n-th point
- mu mean
- Sgm covariance matrix
- Returns
- p p[n] is the probability density function at the n-th point
def evaluateMultiVarGauss(t, mu, Sgm):
This routine must be called for each class c with the same merged test points t .

\\


The interface for the third task should look like this
#
- Parameters
- t coordinates of the test points: t[n,d]
- pXY conditional probabilities: pXY[n,c]
- yEx exact labels of the test points: y[n]

def calculateTestSet(t, pXY, yEx: int):

Here pXY is a matrix whose columns are the output of evaluateMultiVarGauss.
This routine should also print the probabilities and the actual and computed labels
for each test point. It should also count the number of mislabelled points.

To test your routines generate three classes with the following parameters

\begin{eqnarray}
mean = \begin{bmatrix}
0\\
0
\end{bmatrix},  
cov = \begin{bmatrix}
2 & 1\\
1 & 50
\end{bmatrix}, nx = 40, nt = 10
\end{eqnarray}

\begin{eqnarray}
mean = \begin{bmatrix}
7\\
5
\end{bmatrix},  
cov = \begin{bmatrix}
3 & 1\\
1 & 3
\end{bmatrix}, nx = 80, nt = 20
\end{eqnarray}

\begin{eqnarray}
mean = \begin{bmatrix}
-5\\
5
\end{bmatrix},  
cov = \begin{bmatrix}
5 & 2\\
2 & 3
\end{bmatrix}, nx = 20, nt = 5
\end{eqnarray}

## Question 1

Run the code as described above. This will generate quadratic decision
boundaries. This should give a very small number of misclassified test
points.

## Solution

In [ ]:
import numpy as np

def getArtificialData(mean, cov, nx, nt):
    x = np.random.multivariate_normal(mean, cov, nx)
    t = np.random.multivariate_normal(mean, cov, nt)
    mu = np.mean(x, axis=0)
    Sgm = np.cov(x, rowvar=False)
    return x, t, mu, Sgm

def evaluateMultiVarGauss(t, mu, Sgm):
    d = len(mu)
    det_Sgm = np.linalg.det(Sgm)
    inv_Sgm = np.linalg.inv(Sgm)
    p = np.zeros(len(t))
    for i in range(len(t)):
        diff = t[i] - mu
        p[i] = np.exp(-0.5 * np.dot(np.dot(diff, inv_Sgm), diff)) / np.sqrt((2*np.pi)**d * det_Sgm)
    return p

def calculateTestSet(t, pXY, yEx):
    pred_labels = np.argmax(pXY, axis=1)
    misclassified = np.sum(pred_labels != yEx)
    print("Number of misclassified points:", misclassified)
    print("Actual labels:", yEx)
    print("Predicted labels:", pred_labels)
    print("Probabilities:", pXY)

# Generating artificial data and testing
mean1 = np.array([0, 0])
cov1 = np.array([[2, 1], [1, 50]])
nx1 = 40
nt1 = 10

mean2 = np.array([7, 5])
cov2 = np.array([[3, 1], [1, 3]])
nx2 = 80
nt2 = 20

mean3 = np.array([-5, 5])
cov3 = np.array([[5, 2], [2, 3]])
nx3 = 20
nt3 = 5

x1, t1, mu1, Sgm1 = getArtificialData(mean1, cov1, nx1, nt1)
x2, t2, mu2, Sgm2 = getArtificialData(mean2, cov2, nx2, nt2)
x3, t3, mu3, Sgm3 = getArtificialData(mean3, cov3, nx3, nt3)

t_all = np.concatenate((t1, t2, t3))
mu_all = [mu1, mu2, mu3]
Sgm_all = [Sgm1, Sgm2, Sgm3]

pXY_all = np.zeros((len(t_all), 3))
for i in range(3):
    pXY_all[:, i] = evaluateMultiVarGauss(t_all, mu_all[i], Sgm_all[i])

yEx_all = np.concatenate((np.zeros(len(t1)), np.ones(len(t2)), np.full(len(t3), 2)))

calculateTestSet(t_all, pXY_all, yEx_all)


Number of misclassified points: 0
Actual labels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2.]
Predicted labels: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2]
Probabilities: [[6.77402820e-03 5.25888994e-04 1.18218903e-06]
 [5.15573488e-03 1.38885165e-13 1.01159996e-04]
 [8.37237555e-03 3.13890385e-06 1.18381784e-13]
 [3.86260383e-03 1.43010897e-14 5.96795302e-28]
 [1.01583279e-02 1.61418050e-07 1.69129296e-04]
 [1.13990065e-02 1.94711128e-06 3.22357444e-11]
 [8.95125488e-03 1.77527815e-04 5.20515556e-06]
 [5.10033717e-03 4.11071859e-10 1.39346260e-04]
 [9.48723177e-03 9.55687402e-07 1.06533165e-13]
 [2.03791119e-03 1.95288151e-18 2.37518976e-32]
 [8.28838793e-08 2.99843686e-02 2.24909554e-12]
 [1.29885714e-05 3.53386062e-02 9.14014354e-13]
 [2.28791324e-06 1.35935555e-02 6.57434189e-10]
 [7.93612964e-09 2.35856917e-02 1.92929996e-14]
 [5.86124305e-06 3.34874429e-02 2.29719729e-13]
 [1.61183537e-06 1

## Question 2

Modify the code to test tied covariances (i.e., linear decision boundaries).
You can compute the tied covariance matrix by the formula

\begin{eqnarray}
\hat{\Sigma} = \frac{1}{N}\sum_{c}N_c\hat{\Sigma_{c}}
\end{eqnarray}

then you can re-use all previously written routines. Run the code with the
same data as before and compare the number of misclassified points with
the result you got in (1).

## Solution

In [ ]:
import numpy as np

def getArtificialData(mean, cov, nx, nt):
    x = np.random.multivariate_normal(mean, cov, nx)
    t = np.random.multivariate_normal(mean, cov, nt)
    mu = np.mean(x, axis=0)
    Sgm = np.cov(x, rowvar=False)
    return x, t, mu, Sgm

def calculateTiedCovariance(data):
    N = len(data)
    mu = np.mean(data, axis=0)
    Sgm_tied = np.zeros((len(mu), len(mu)))  # Initialize tied covariance matrix
    for x in data:
        diff = x - mu
        Sgm_tied += np.outer(diff, diff)
    Sgm_tied /= N
    return Sgm_tied

def evaluateMultiVarGauss(t, mu, Sgm):
    d = len(mu)
    det_Sgm = np.linalg.det(Sgm)
    inv_Sgm = np.linalg.inv(Sgm)
    p = np.zeros(len(t))
    for i in range(len(t)):
        diff = t[i] - mu
        p[i] = np.exp(-0.5 * np.dot(np.dot(diff, inv_Sgm), diff)) / np.sqrt((2*np.pi)**d * det_Sgm)
    return p

def calculateTestSet(t, pXY, yEx):
    pred_labels = np.argmax(pXY, axis=1)
    misclassified = np.sum(pred_labels != yEx)
    print("Number of misclassified points:", misclassified)
    print("Actual labels:", yEx)
    print("Predicted labels:", pred_labels)
    print("Probabilities:", pXY)

# Generating artificial data and testing
mean1 = np.array([0, 0])
cov1 = np.array([[2, 1], [1, 50]])
nx1 = 40
nt1 = 10

mean2 = np.array([7, 5])
cov2 = np.array([[3, 1], [1, 3]])
nx2 = 80
nt2 = 20

mean3 = np.array([-5, 5])
cov3 = np.array([[5, 2], [2, 3]])
nx3 = 20
nt3 = 5

x1, t1, mu1, _ = getArtificialData(mean1, cov1, nx1, nt1)
x2, t2, mu2, _ = getArtificialData(mean2, cov2, nx2, nt2)
x3, t3, mu3, _ = getArtificialData(mean3, cov3, nx3, nt3)

t_all = np.concatenate((t1, t2, t3))
mu_all = [mu1, mu2, mu3]
x_all = [x1, x2, x3]

# Calculate tied covariance matrix
Sgm_tied = calculateTiedCovariance(np.concatenate(x_all))

pXY_all = np.zeros((len(t_all), 3))
for i in range(3):
    pXY_all[:, i] = evaluateMultiVarGauss(t_all, mu_all[i], Sgm_tied)

yEx_all = np.concatenate((np.zeros(len(t1)), np.ones(len(t2)), np.full(len(t3), 2)))

calculateTestSet(t_all, pXY_all, yEx_all)


Number of misclassified points: 2
Actual labels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2.]
Predicted labels: [0 0 0 0 0 0 2 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2]
Probabilities: [[8.76155385e-04 1.20229460e-04 2.71916743e-05]
 [6.54896207e-03 2.33243450e-03 3.36197513e-03]
 [4.90538754e-03 2.93283196e-03 3.30998908e-03]
 [1.68143813e-03 2.83587181e-04 7.46329249e-05]
 [6.14610296e-05 2.17814884e-06 1.17546294e-06]
 [5.23385269e-03 2.13106047e-03 4.22839879e-03]
 [4.53621836e-04 6.71206360e-04 1.35883530e-03]
 [1.99627048e-03 1.94104169e-04 1.55291401e-04]
 [2.45852098e-03 2.86736742e-03 2.25642341e-03]
 [3.77963162e-03 8.11769555e-04 3.22672837e-04]
 [6.48133569e-04 5.56505633e-03 7.87887339e-05]
 [1.32580526e-03 6.67263666e-03 1.06343592e-04]
 [2.24125424e-03 7.06519679e-03 2.69893096e-04]
 [1.67087613e-03 6.69939826e-03 2.84255643e-04]
 [3.52850811e-03 6.63282904e-03 5.38414103e-04]
 [1.84424462e-03 7

## Question 3

Test the methodology on the iris data set, that was mentioned in the lecture.
You can import it with the following commands
from sklearn.datasets import load_iris
iris = load_iris()
the petal/sepal dimensions are in the array iris.data and the correspond-
ing labels are in the array iris.target. You will notice that the first 50
entries have label 0, followed by 50 entries with label 1 and 50 entries with
label 2. Use the first 40 entries in each class as data points and the next
10 entries as test points. You have to write a new routine getIrisData()
with similar functionality as getArtificialData(). If you have done the
implementation of evaluateMultiVarGauss() and calculateTestSet()
right, then you will be able to run these routines with the iris data set as
well. Again, count the number of misclassified items in the test set.

## Solution

### Without tied covariance

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

def getIrisData():
    iris = load_iris()
    data = iris.data
    target = iris.target

    # Extracting first 40 entries from each class as data points
    x1 = data[target == 0][:40]
    x2 = data[target == 1][:40]
    x3 = data[target == 2][:40]

    # Extracting next 10 entries from each class as test points
    t1 = data[target == 0][40:50]
    t2 = data[target == 1][40:50]
    t3 = data[target == 2][40:50]

    # Calculate mean and covariance for each class
    mu1 = np.mean(x1, axis=0)
    mu2 = np.mean(x2, axis=0)
    mu3 = np.mean(x3, axis=0)
    Sgm1 = np.cov(x1, rowvar=False)
    Sgm2 = np.cov(x2, rowvar=False)
    Sgm3 = np.cov(x3, rowvar=False)

    return [x1, x2, x3], [t1, t2, t3], [mu1, mu2, mu3], [Sgm1, Sgm2, Sgm3]

# Function to evaluate multivariate Gaussian distribution
def evaluateMultiVarGauss(t, mu, Sgm):
    d = len(mu)
    det_Sgm = np.linalg.det(Sgm)
    inv_Sgm = np.linalg.inv(Sgm)
    p = np.zeros(len(t))
    for i in range(len(t)):
        diff = t[i] - mu
        p[i] = np.exp(-0.5 * np.dot(np.dot(diff, inv_Sgm), diff)) / np.sqrt((2*np.pi)**d * det_Sgm)
    return p

# Function to calculate misclassified points in the test set
def calculateTestSet(t, pXY, yEx):
    pred_labels = np.argmax(pXY, axis=1)
    misclassified = np.sum(pred_labels != yEx)
    print("Number of misclassified points:", misclassified)
    print("Actual labels:", yEx)
    print("Predicted labels:", pred_labels)
    print("Probabilities:", pXY)

# Get Iris data and test
x_all, t_all, mu_all, Sgm_all = getIrisData()

# Calculate probabilities for each class
pXY_all = np.zeros((len(np.concatenate(t_all)), 3))
for i in range(3):
    pXY_all[:, i] = evaluateMultiVarGauss(np.concatenate(t_all), mu_all[i], Sgm_all[i])

# Set actual labels for test set
yEx_all = np.concatenate((np.zeros(len(t_all[0])), np.ones(len(t_all[1])), np.full(len(t_all[2]), 2)))

# Calculate and print misclassified points in the test set
calculateTestSet(np.concatenate(t_all), pXY_all, yEx_all)


Number of misclassified points: 0
Actual labels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2.]
Predicted labels: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
Probabilities: [[7.81435194e+000 5.88902311e-023 9.35547567e-036]
 [2.93126805e-003 1.73532119e-012 4.79357840e-026]
 [2.45619345e+000 1.70664995e-019 1.93466684e-029]
 [4.81174613e-003 3.58520997e-018 6.64854132e-029]
 [9.87470311e-002 6.31867093e-021 8.39767713e-030]
 [3.45674864e+000 2.01425700e-016 2.61432439e-029]
 [3.52940179e+000 4.57657405e-026 8.55244329e-036]
 [9.29964603e+000 5.79339655e-019 1.29804288e-029]
 [1.12524756e+001 1.12468803e-025 4.38094192e-038]
 [1.57261747e+001 9.24900533e-021 1.06527756e-033]
 [7.13615519e-074 2.40751868e-001 1.31807440e-002]
 [3.98144515e-086 2.44508165e+000 1.83458137e-002]
 [8.29902570e-060 3.92526536e+000 3.08060272e-004]
 [3.39890017e-036 3.67453462e-001 5.59130953e-006]
 [1.93728765e-069 3.33068754e+000 9.94926107e-003

### With tied covariance

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

def getIrisData():
    iris = load_iris()
    data = iris.data
    target = iris.target

    # Extracting first 40 entries from each class as data points
    x1 = data[target == 0][:40]
    x2 = data[target == 1][:40]
    x3 = data[target == 2][:40]

    # Extracting next 10 entries from each class as test points
    t1 = data[target == 0][40:50]
    t2 = data[target == 1][40:50]
    t3 = data[target == 2][40:50]

    # Calculate mean and tied covariance for each class
    mu1 = np.mean(x1, axis=0)
    mu2 = np.mean(x2, axis=0)
    mu3 = np.mean(x3, axis=0)
    x_all = [x1, x2, x3]

    # Calculate tied covariance matrix
    Sgm_tied = calculateTiedCovariance(np.concatenate(x_all))

    return x_all, [t1, t2, t3], [mu1, mu2, mu3], Sgm_tied

# Function to calculate tied covariance matrix
def calculateTiedCovariance(data):
    N = len(data)
    mu = np.mean(data, axis=0)
    Sgm_tied = np.zeros_like(np.cov(data.T))
    for x in data:
        diff = x - mu
        Sgm_tied += np.outer(diff, diff)
    Sgm_tied /= N
    return Sgm_tied

# Function to evaluate multivariate Gaussian distribution
def evaluateMultiVarGauss(t, mu, Sgm):
    d = len(mu)
    det_Sgm = np.linalg.det(Sgm)
    inv_Sgm = np.linalg.inv(Sgm)
    p = np.zeros(len(t))
    for i in range(len(t)):
        diff = t[i] - mu
        p[i] = np.exp(-0.5 * np.dot(np.dot(diff, inv_Sgm), diff)) / np.sqrt((2*np.pi)**d * det_Sgm)
    return p

# Function to calculate misclassified points in the test set
def calculateTestSet(t, pXY, yEx):
    pred_labels = np.argmax(pXY, axis=1)
    misclassified = np.sum(pred_labels != yEx)
    print("Number of misclassified points:", misclassified)
    print("Actual labels:", yEx)
    print("Predicted labels:", pred_labels)
    print("Probabilities:", pXY)

# Get Iris data and test
x_all, t_all, mu_all, Sgm_tied = getIrisData()

# Calculate probabilities for each class
pXY_all = np.zeros((len(np.concatenate(t_all)), 3))
for i in range(3):
    pXY_all[:, i] = evaluateMultiVarGauss(np.concatenate(t_all), mu_all[i], Sgm_tied)

# Set actual labels for test set
yEx_all = np.concatenate((np.zeros(len(t_all[0])), np.ones(len(t_all[1])), np.full(len(t_all[2]), 2)))

# Calculate and print misclassified points in the test set
calculateTestSet(np.concatenate(t_all), pXY_all, yEx_all)


Number of misclassified points: 2
Actual labels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2.]
Predicted labels: [0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2]
Probabilities: [[4.54916324e-01 6.58986143e-02 2.78920213e-02]
 [1.24527684e-03 2.63726150e-03 9.69484255e-05]
 [2.31087777e-01 6.49994770e-02 1.58943673e-02]
 [1.63846718e-01 2.65905796e-02 2.05158568e-02]
 [1.46555046e-01 1.63812299e-02 1.66534022e-02]
 [2.07576164e-01 9.91956534e-02 1.52586758e-02]
 [1.88975549e-01 1.81641187e-02 1.22466739e-02]
 [3.58829547e-01 1.10856704e-01 2.46336945e-02]
 [4.56337852e-01 5.20381657e-02 2.49171085e-02]
 [5.36601567e-01 1.37075854e-01 3.08747759e-02]
 [8.06906631e-03 6.96682396e-02 2.25791512e-02]
 [8.00197241e-02 3.17353188e-01 2.75393593e-01]
 [7.71178286e-02 5.06940312e-01 1.34417121e-01]
 [1.61141756e-02 1.23824956e-01 1.62223083e-02]
 [6.40723862e-02 3.68718217e-01 1.68166660e-01]
 [4.17196752e-02 1.27147585e-01 8.95389307e